# Using the model to cluster songs according to their genre

### Loading the model

In [3]:
import torch
import torch.nn as nn
import deeplay as dl
import deeptrack as dt
import os

vae = dl.VariationalAutoEncoder(input_size=(256,647),
    latent_dim=20, channels=[32, 64],
    reconstruction_loss=torch.nn.BCELoss(reduction="mean"), beta=1e-3
).create()
vae.load_state_dict(torch.load("vae.pth"))
vae.eval()

#Preprocess data
data_dir = os.path.expanduser("./trainImages")

#Load image files using ImageFolder
trainFiles = dt.sources.ImageFolder(root=data_dir)

class CropWidth:
    def __init__(self, target_width):
        self.target_width = target_width

    def __call__(self, x: torch.Tensor):
        # assuming input shape [C, H, W]
        return x[..., :self.target_width]
    
image_pip = (dt.LoadImage(trainFiles.path) >> dt.NormalizeMinMax()
             >> dt.MoveAxis(2, 0) >> dt.pytorch.ToTensor(dtype=torch.float) >> CropWidth(1904))

train_dataset = dt.pytorch.Dataset(image_pip & image_pip, inputs=trainFiles)
train_loader = dl.DataLoader(train_dataset, batch_size=1, shuffle=True)

from Image2Sound import *
from torchvision.utils import save_image
for image, _ in train_loader:
    mu, _ = vae.encode(image)
    image = vae.decode(mu).clone().detach().squeeze(0)
    break
"""
latentSpaceImage = vae.encode(img_tensor.unsqueeze(0))
print(latentSpaceImage.shape)
z = 255*torch.stack(latentSpaceImage)
print(z)
"""
iamge = image.float()
print(min(image))
print(max(image))
class conf:
        sampling_rate = 44100
        duration = 30
        hop_length = 694
        fmin = 20
        fmax = sampling_rate // 2
        n_mels = 128
        n_fft = n_mels * 20
        samples = sampling_rate * duration
save_image(image, 'test_sample.jpg')
audio = Image2Sound('test_sample.jpg', conf)
SaveAudio(audio,os.getcwd(),"test_sample.mp3")



tensor([[0.4822, 0.5476, 0.5545,  ..., 0.5704, 0.5642, 0.5303],
        [0.4887, 0.5996, 0.6161,  ..., 0.6467, 0.6339, 0.5812],
        [0.5008, 0.6302, 0.6457,  ..., 0.6841, 0.6746, 0.6041],
        ...,
        [0.0060, 0.0208, 0.0243,  ..., 0.0193, 0.0197, 0.0484],
        [0.0056, 0.0182, 0.0213,  ..., 0.0169, 0.0170, 0.0440],
        [0.0231, 0.0171, 0.0213,  ..., 0.0203, 0.0205, 0.0673]])
tensor([[0.4822, 0.5476, 0.5545,  ..., 0.5704, 0.5642, 0.5303],
        [0.4887, 0.5996, 0.6161,  ..., 0.6467, 0.6339, 0.5812],
        [0.5008, 0.6302, 0.6457,  ..., 0.6841, 0.6746, 0.6041],
        ...,
        [0.0060, 0.0208, 0.0243,  ..., 0.0193, 0.0197, 0.0484],
        [0.0056, 0.0182, 0.0213,  ..., 0.0169, 0.0170, 0.0440],
        [0.0231, 0.0171, 0.0213,  ..., 0.0203, 0.0205, 0.0673]])
[[118 145 143 ... 149 145 136]
 [126 155 155 ... 162 158 149]
 [132 162 165 ... 175 170 160]
 ...
 [  3   4   6 ...   4   6   9]
 [  3   5   6 ...   4   7  10]
 [  3   5   7 ...   4   8  11]]


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab